## Variables de entorno

In [0]:

dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
dbutils.widgets.text("STORAGE_NAME", "adlsdatalakev2")
dbutils.widgets.text("CONTAINER", "raw")
dbutils.widgets.text("CATALOG", "principal")
dbutils.widgets.text("SCHEMA", "bronze")

In [0]:

storage_name = dbutils.widgets.get("STORAGE_NAME")
container = dbutils.widgets.get("CONTAINER")
catalog =  dbutils.widgets.get("CATALOG")
schema =  dbutils.widgets.get("SCHEMA")


## Rutas

In [0]:
padre = f"abfss://{container}@{storage_name}.dfs.core.windows.net/"

rutaOrders =  f"{padre}orders.csv"
rutaProducts =  f"{padre}products.csv"
rutaOrderDetails  =  f"{padre}order-details.csv"

## Definicion de estructuras


### Products

In [0]:
products_schema = StructType([
    StructField("ProductID", IntegerType(), True),
    StructField("ProductName", StringType(), True),
    StructField("QuantityPerUnit", StringType(), True),
    StructField("UnitPrice", DoubleType(), True)
])

df_products = (
    spark.read
        .format("csv")
        .schema(products_schema)
        .option("header", True)
        .csv(rutaProducts)
)


### Orders

In [0]:
orders_schema = StructType([
    StructField("OrderID", IntegerType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("EmployeeID", IntegerType(), True),
    StructField("OrderDate", DateType(), True),
    StructField("ShipCountry", StringType(), True)
])

df_orders = (
    spark.read
        .format("csv")
        .schema(orders_schema)
        .option("header", True)
        .csv(rutaOrders)
)


### Orders details

In [0]:
order_details_schema = StructType([
    StructField("OrderID", IntegerType(), True),
    StructField("ProductID", IntegerType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Discount", DoubleType(), True)
])

df_order_details = (
    spark.read
        .format("csv")
        .schema(order_details_schema)
        .option("header", True)
        .csv(rutaOrderDetails)
)


G


## Guardado

In [0]:
df_products.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.products")
df_orders.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.orders")
df_order_details.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.order_details")